In [3]:
import numpy as np
np.random.seed(0)
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import regex as re
import nltk
from nltk.corpus import stopwords
from keras.layers import Dense, Input, Dropout, LSTM, Activation,Flatten,Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
np.random.seed(1)

In [4]:
df=pd.read_csv('train.csv')
df1=pd.read_csv('test.csv')

In [5]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [6]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [7]:
X = []
sentences = list(df['tweet'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [8]:
print(X)

[' fingerprint Pregnancy Test https goo gl MfQV android apps beautiful cute health igers iphoneonly iphonesia iphone', 'Finally transparant silicon case Thanks to my uncle yay Sony Xperia sonyexperias http instagram com YGEt JC JM ', 'We love this Would you go talk makememories unplug relax iphone smartphone wifi connect http fb me LsUpCu', 'I wired know m George was made that way iphone cute daventry home http instagr am Li ujS ', 'What amazing service Apple won even talk to me about question have unless pay them for their stupid support ', 'iPhone software update fucked up my phone big time Stupid iPhones', 'Happy for us instapic instadaily us sony xperia xperiaZ https instagram com z qGfWlvj ', 'New Type charger cable UK http www ebay co uk itm bay Amazon etsy New Year Rob Cross Toby Young EVEMUN McMafia Taylor SPECTRE NewYear Starting recipes technology SamsungGalaxyS iPhoneX pic twitter com PjIwq WtC', 'Bout to go shopping again listening to music iphone justme music likeforlike f

In [9]:
y=df['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [11]:
tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [12]:
print(X_train)

[[141, 459, 6003, 922, 6004, 6005, 1027, 6006, 6007, 6008, 6009, 6010, 6011, 6012, 3894, 3895, 881, 2960, 9, 1, 3896, 459, 3897, 628, 1268, 6013, 963, 2043, 2044, 3898, 203, 6014, 11, 3, 6015, 6016], [56, 341, 85, 8, 7, 21, 2961, 6017, 37, 2962, 802, 7, 6018, 2, 145, 90, 6019, 3899, 6020], [1096, 76, 7, 1759, 3900, 24, 163, 25, 7, 1181, 142, 8, 1097, 7, 163, 233, 37, 7, 163, 964], [6021, 6022, 6, 71, 842, 1551, 6023, 83, 9, 6024, 6025, 2, 22, 19, 6026], [35, 609, 191, 1552, 4, 733, 310, 592, 4, 191, 6027, 609, 1098, 1760, 4], [679, 6028, 104, 239, 83, 34, 408, 1269, 258, 1, 3901, 77, 6029, 54, 2, 22, 19, 6030, 2045], [18, 157, 160, 803, 151, 1, 1270, 1099, 923, 1182, 963, 88, 25, 16, 28, 6, 3, 6031, 6032], [7, 6033, 26, 7, 311, 24, 6034, 2, 185, 10, 6035, 62, 51, 77, 153, 123, 89, 33, 1, 81, 79], [31, 57, 659, 92, 69, 290, 10, 72, 6036, 6037, 6038, 6039, 495, 6040, 304, 71, 51, 660, 6041, 2046, 6042, 6043, 965, 2410, 680, 3902, 3902, 21, 804, 2, 154, 105, 6044, 30, 11, 3, 510, 6045, 60

In [11]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 50

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [12]:
embeddings_dictionary = dict()
glove_file = open('glove.6B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [13]:
embedding_matrix = np.zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [14]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], trainable=False)
model.add(embedding_layer)
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
#model.add(Activation('softmax'))

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          971300    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,194,661
Trainable params: 223,361
Non-trainable params: 971,300
________________________________________

In [17]:
model.fit(X_train,y_train,epochs=50,batch_size=1000,shuffle=True)

Epoch 1/50
6336/6336 [==============================] - 13s 2ms/step - loss: 0.6546 - accuracy: 0.6872
Epoch 2/50
6336/6336 [==============================] - 12s 2ms/step - loss: 0.5097 - accuracy: 0.7568
Epoch 3/50
6336/6336 [==============================] - 13s 2ms/step - loss: 0.3873 - accuracy: 0.8204
Epoch 4/50
6336/6336 [==============================] - 14s 2ms/step - loss: 0.3526 - accuracy: 0.8412
Epoch 5/50
6336/6336 [==============================] - 15s 2ms/step - loss: 0.3217 - accuracy: 0.8647
Epoch 6/50
6336/6336 [==============================] - 14s 2ms/step - loss: 0.2937 - accuracy: 0.8731
Epoch 7/50
6336/6336 [==============================] - 13s 2ms/step - loss: 0.2830 - accuracy: 0.8767
Epoch 8/50
6336/6336 [==============================] - 14s 2ms/step - loss: 0.2749 - accuracy: 0.8816
Epoch 9/50
6336/6336 [==============================] - 13s 2ms/step - loss: 0.2714 - accuracy: 0.8824
Epoch 10/50
6336/6336 [==============================] - 14s 2ms/step - l

In [18]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
print(X_test.shape)
preds=model.predict(X_test)
preds=(preds>0.5).astype(int)
print(confusion_matrix(preds,y_test))
print(classification_report(preds,y_test))
print(roc_auc_score(preds,y_test))

(1584, 50)
[[1043  110]
 [ 109  322]]
              precision    recall  f1-score   support

           0       0.91      0.90      0.90      1153
           1       0.75      0.75      0.75       431

    accuracy                           0.86      1584
   macro avg       0.83      0.83      0.83      1584
weighted avg       0.86      0.86      0.86      1584

0.8258482361156108


In [20]:
model_json=model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [24]:
print(model_json)

{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "Embedding", "config": {"name": "embedding_1", "trainable": false, "batch_input_shape": [null, null], "dtype": "float32", "input_dim": 19426, "output_dim": 50, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": null}}, {"class_name": "LSTM", "config": {"name": "lstm_1", "trainable": true, "dtype": "float32", "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "units": 128, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "VarianceScaling", "config": {"scale": 1.0, "mode": "fan_avg", "distribution": "uniform", "seed": null}}, "recurrent_initializer": {"class_name": "Orthogonal", "config": 

In [23]:
model.save_weights('classifier.h5')

In [22]:
X = []
sentences = list(df1['tweet'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [23]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)

In [24]:
X=tokenizer.texts_to_sequences(X)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_test = pad_sequences(X, padding='post', maxlen=maxlen)

preds=model.predict(X_test)

In [25]:
print((preds>0.5).astype(int))

[[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]


In [26]:
preds=(preds>0.5).astype(int)
preds=preds.reshape(preds.shape[0]*preds.shape[1])

In [27]:
data=pd.DataFrame({'id':df1.id,'label':preds})

In [28]:
data.to_csv('sent_submission.csv',index=False)